In [2]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import h5py
from math import pi
from scipy.stats import zscore

from analysis.config import birds, h5_path_dict, pickle_dir
from analysis.ExpData import ExpData
from analysis.utils import get_fr

# Collect population patterns and save into pickle file
Hop patterns are (hops, neurs, frames) size

### Parameters

In [3]:
window = int(20*2)
fr_smoothing_window = int(20*0.25)
l2_normalization = True
pfile_name = "population_patterns_checks.p"

### Data formatting

In [ ]:
data = {}
for key in h5_path_dict.keys():
    for fil in range(len(h5_path_dict[key])):
        fil_string = h5_path_dict[key][fil].as_posix()
        print(fil_string)
        f = h5py.File(fil_string, 'r')
        exp_data = ExpData(f)
        data[fil_string] = {}
        c_hops, r_hops, ch_hops, noncrch_hops = exp_data.get_crch_hops()
        import pdb; pdb.set_trace()
        hop_windows = exp_data.get_hop_windows(window)
        fr = get_fr(exp_data.spikes, fr_smoothing_window)
        if l2_normalization:
            scaling = np.square(np.linalg.norm(fr, axis=1))
            #scaling = np.linalg.norm(fr, axis=1)
            fr /= scaling[:, None]
            fr[np.isnan(fr)] = 0
        hop_patterns = np.zeros(
            (hop_windows.shape[0], exp_data.num_neurs, window*2 + 1)
            )
        for i, hop in enumerate(hop_windows):
            hop_pattern = fr[:, hop]
            hop_pattern[:, hop == -1] = np.nan
            hop_patterns[i] = hop_pattern            
        data[fil_string]['c_hops'] = c_hops
        data[fil_string]['r_hops'] = r_hops
        data[fil_string]['ch_hops'] = ch_hops
        data[fil_string]['noncrch_hops'] = noncrch_hops
        data[fil_string]['fr'] = fr
        data[fil_string]['hop_patterns'] = hop_patterns
        data[fil_string]['hop_times'] = hop_windows
with open(pickle_dir / pfile_name, 'wb') as f: 
    pickle.dump(data, f)